In [ ]:
!pip install datasets

In [ ]:
!pip install openai

In [ ]:
from datasets import load_dataset

In [ ]:
import os
import openai

In [ ]:
dataset = load_dataset("boolq")
shuffled_dataset = dataset.shuffle(seed=7)

In [ ]:
prompt_true = []
prompt_false = []
true_count = 0
false_count = 0

for i in range(len(shuffled_dataset['train'])):
  if shuffled_dataset['train'][i]['answer'] == True and true_count < 4:
    prompt_true.append(shuffled_dataset['train'][i])
    true_count += 1
  elif shuffled_dataset['train'][i]['answer'] == False and false_count < 4:
    prompt_false.append(shuffled_dataset['train'][i])
    false_count += 1

In [ ]:
prompt = ""

for i in range(4):
  prompt += "question: " + prompt_true[i]['question'] + "\n" + "answer: " + str(prompt_true[i]['answer']) + "\n"
  prompt += "question: " + prompt_false[i]['question'] + "\n" + "answer: " + str(prompt_false[i]['answer']) + "\n"

In [ ]:
print(prompt)

question: can i raise a wolf as a pet
answer: True
question: can you buy a creme egg all year round
answer: False
question: did kobe bryant started his first year in the nba
answer: True
question: has any team not won the world series
answer: False
question: has any golfer won all 4 majors in 1 year
answer: True
question: was the quiet man filmed in black and white
answer: False
question: is night at the museum based on a book
answer: True
question: is the small intestine part of the upper gi tract
answer: False



In [ ]:
validation = []

for i in range(30):
  validation.append(shuffled_dataset['validation'][i])

In [ ]:
validation[0]

{'question': 'do you get money for nobel peace prize',
 'answer': True,
 'passage': 'Between 1901 and 2017, the Nobel Prizes including the Economic Prizes were awarded 585 times to 923 people and organizations. With some receiving the Nobel Prize more than once, this makes a total of 24 organizations, and 892 individuals. The prize ceremonies take place annually in Stockholm, Sweden (with the exception of the peace prize, which is held in Oslo, Norway). Each recipient, or laureate, receives a gold medal, a diploma, and a sum of money that has been decided by the Nobel Foundation. (As of 2017, each prize is worth 9,000,000 SEK, or about US$1,110,000, €944,000, £836,000 or ₹72,693,900.) Medals made before 1980 were struck in 23 carat gold, and later in 18 carat green gold plated with a 24 carat gold coating.'}

In [ ]:
openai.api_key = ""

In [ ]:
responses = []

In [ ]:
for v in validation:
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt= prompt + "question: " + v['question'] + "\n" + "answer: ",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  responses.append(response)

In [ ]:
correct_pair = 0

for i in range(30):
  if str(validation[i]["answer"]) == responses[i]["choices"][0]["text"][1:]:
    correct_pair += 1

print(f'Few-shot accuracy is', correct_pair/30)

Few-shot accuracy is 0.8333333333333334


### Q5-5

In [ ]:
!pip install transformers

In [ ]:
responses_bloomz = []

prompt_bloomz = ""

for i in range(4):
  prompt_bloomz += "question: " + prompt_true[i]['question'] + " " + "answer: " + str(prompt_true[i]['answer']) + " "
  prompt_bloomz += "question: " + prompt_false[i]['question'] + " " + "answer: " + str(prompt_false[i]['answer']) + " "

In [ ]:
import requests

API_URL = "https://api-inference.huggingface.co/models/bigscience/bloomz"
headers = {"Authorization": f""}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
for v in validation:
  tmp = prompt_bloomz + "question: " + v['question'] + " " + "answer:"
  output = query({
    "inputs": tmp,
  })
  responses_bloomz.append(output)

In [ ]:
for i in range(30):
  print(responses_bloomz[i])

[{'generated_text': 'question: can i raise a wolf as a pet answer: True question: can you buy a creme egg all year round answer: False question: did kobe bryant started his first year in the nba answer: True question: has any team not won the world series answer: False question: has any golfer won all 4 majors in 1 year answer: True question: was the quiet man filmed in black and white answer: False question: is night at the museum based on a book answer: True question: is the small intestine part of the upper gi tract answer: False question: do you get money for nobel peace prize answer: False'}]
[{'generated_text': 'question: can i raise a wolf as a pet answer: True question: can you buy a creme egg all year round answer: False question: did kobe bryant started his first year in the nba answer: True question: has any team not won the world series answer: False question: has any golfer won all 4 majors in 1 year answer: True question: was the quiet man filmed in black and white answer

In [30]:
correct_pair_bloomz= 0

for i in range(30):
  if str(validation[i]["answer"]) == responses_bloomz[i][0]['generated_text'][-5:] or str(validation[i]["answer"]) == responses_bloomz[i][0]['generated_text'][-4:]:
    correct_pair_bloomz += 1

print(f'Few-shot accuracy is', correct_pair_bloomz/30)

Few-shot accuracy is 0.3
